### PIECES LEFT TO DO:

* __Make sure you also are grabbing any file that isn't an MP3__
    * __This may mean grabbing all files and putting them in a table first, then going in and checking their tags__
    * __Or pull everything in, but have new columns for dirpath, dirname, filename so that you can grab all rows with the same dirpath as opposed to just grabbing things that have `.mp3` in the full dirpath__

* __Go through `deds` and build table of correct data for it__

    * __Bonus points if you can find an API or something to hit__

* __Perform the actual file move once everything is up to date__



-----------------



This project will let me see if I can:

* Read id3 tags in Python

* Create a new directory with music files organized by [main]Artist/Album/Song
   
  * as opposed to [Song] Artists/Album/Single Song
    
I am so tired of Apple's music file directory schema. It makes it impossible to listen to a full album from a place like VLC player because each song that has a feature or colab is given its own directory tree instead of just being in the album folder. It pisses me off to no end and I am now going to try to write a script to read everything, organize it, and rewrite the directory.

There is a good chance that this will be harder than I want it to be. Meh. Should be fun!

In [55]:
## https://eyed3.readthedocs.io/en/latest/
from os import listdir
from os.path import isfile, join
from os import walk
import os
import shutil


import eyed3 as id3
from mutagen.mp4 import MP4

import pandas as pd

pd.set_option('display.width', 1000)



dests = pd.DataFrame(columns=['file_origin', 'Artist', 'Album', 'New_Loc', 'New_dir'])

## Will want to either loop through all files in a directory tree or generate a list of files in a target directory
### Something like a list of every file, go through each one, grab id3 info
### Then build a theoretical directory tree into a table [?]
### Run a file writer that iterates through each song in the table and creates the appropriate directories in a secondary location

### IF COMPILATION ALBUM/SOUNDTRACK: STORE WITHOUT ARTIST FOLDER, i.e City Lounge 1 will be a folder with songs at the artist level


# mypath = "L:\DeathStar\Music\Music"
# newpath = "D:\Music"

mypath = "/home/james/Music"
newpath = "/home/james/Music/Cleaned"

#onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
#print(listdir(mypath))

files = pd.DataFrame(columns=['dir_path',
                              'dir_name',
                              'file_name',
                              'full_path',
                              'new_dir_path',
                              'new_dir_name'])

####### THIS IS WHERE YOU WOULD BE MAKING THE ADDITION TO CATCH THUMBNAILS, ETC. #########


for (dirpath, dirnames, filenames) in walk(mypath):
    if filenames:
        for i in filenames:
            dp = dirpath
            dn = dirnames
            fn = i
            fp = os.path.join(dirpath, i)
            nrow = pd.DataFrame({'dir_path': [dp],
                                 'dir_name': [dn],
                                 'file_name': [fn],
                                 'full_path': [fp],
                                 'new_dir_path': ['NA'],
                                 'new_dir_name': ['NA']
                                 }
                               )
            files = files.append(nrow)
print("Files Read!\nFound {} files".format(len(files)))




##########################################################################################################
####### GOING TO HAVE TO CHANGE THIS SO THAT IT'S PULLING FROM THE [NEW] 'FILES' DATAFRAME AND FINALLY ###
####### ADDING A NEW COLUMN OF THE NEW LOCS TO 'FILES' USING THE EXISTING COLUMNS TO BUILD ###############
##########################################################################################################

## This will allow us to keep files like thumbnails 


### VVVVVVVVV CHANGE VVVVVVVVV ###
deds = []
cnt = 0
for song in files.full_path:
    print(song)
    if song.endswith(".mp3"):
        try:
            test_file = id3.load(song)
        except Exception as err:
            print("{}\nError: {}".format(song, err))
            deds += [song]
        else:
            try:
                tit = test_file.tag.title
                song_file = tit + ".mp3"
                art = test_file.tag.artist
                alb = test_file.tag.album
                ndir = os.path.join(newpath,art,alb)
                npth = os.path.join(newpath,art,alb,song_file)
                
            except Exception as err:
                print("Could not get album or artist info for {}\nError: {}".format(song,err))
                deds += [song]
                continue
            else:                
                if art and alb:
                    row = pd.DataFrame(data={'file_origin': [song], 
                                             'Artist': [art], 
                                             'Album': [alb], 
                                             'New_Loc': [npth],
                                            'New_Dir': [ndir]}
                                      )
                    dests = dests.append(row)
                else:
                  #  print("Could not get id3 tag info for {}".format(song))
                    deds += [song]
    elif song.endswith(".m4a"):
        try:
            test_file = MP4(song)
        except Exception as err:
            print("{}\nError: {}".format(song, err))
            deds += [song]
        else:
            try:
                tit = test_file.tags.get('\xa9nam')[0]
                song_file = tit + ".m4a"
                art = test_file.tags.get('\xa9ART')[0]
                alb = test_file.tags.get('\xa9alb')[0]
                ndir = os.path.join(newpath,art,alb)
                npth = os.path.join(newpath,art,alb,song_file)
                
            except Exception as err:
                print("Could not get album or artist info for {}\nError: {}".format(song,err))
                deds += [song]
                continue
            else:                
                if art and alb:
                    row = pd.DataFrame(data={'file_origin': [song], 
                                             'Artist': [art], 
                                             'Album': [alb], 
                                             'New_Loc': [npth],
                                            'New_Dir': [ndir]}
                                      )
                    dests = dests.append(row)
                else:
                    print("Could not get id3 tag info for {}".format(song))
                    deds += [song]

                #shutil.copyfile2(song, os.path.join(art,alb))
    cnt += 1
    print("Finished with {} of {}".format(cnt, len(files.full_path)))
            #print('-------------------')
            

## Save dests as a csv so that you don't have to rerun this on the giant sample of ~18k songs?
            
            
            

#################################################
#                                               #
#                    NOTES                      #
#                                               #
#################################################

   
# There will just have to be some manual work, darn



# Before copying, test to see if the full fiepath exists before copying
## might help roust out duplicates

# OR build in logic:
#  -create csv/dataframe of all suggested/scraped id3 tags
#  -try to see if there are albums with more than one artist
    # something like for each album, count number of unique artsits
        # This will also require a bit of oversight as some albums will be tricky
#  -if more than one
    # find out if it is a compilation album (manual maybe)
    # if it's just a random artist on the album and have the directory be the majority artist on the album
        # This may not even be neccessary but it's not a bad idea



#################################################


## Load the song file
#test_file = id3.load("/home/james/Music/Ghostface Killah & Amy Winehouse/More Fish/14 You Know I'm No Good.mp3")

## Primary Folder: Artist
#print(test_file.tag.artist)

## Secondary Folder: Album
#print(test_file.tag.album)






/home/james/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:6201: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  sort=sort)
FrameHeader: Illegal Frame ID: b'CM1\x00'


Files Read!
Found 328 files
/home/james/Music/Ghostface Killah Feat. Trife Da God & Mr/More Fish/05 Good.mp3
Finished with 1 of 328
/home/james/Music/Ghostface Killah/Supreme Clientele/12 We Made It.mp3
Finished with 2 of 328
/home/james/Music/Ghostface Killah/Supreme Clientele/18 Cher Chez La Ghost.mp3
Finished with 3 of 328
/home/james/Music/Ghostface Killah/Supreme Clientele/02 Nutmeg.mp3
Finished with 4 of 328
/home/james/Music/Ghostface Killah/Supreme Clientele/15 Malcolm.mp3
Finished with 5 of 328
/home/james/Music/Ghostface Killah/Supreme Clientele/16 Who Would You Fuck.mp3
Finished with 6 of 328
/home/james/Music/Ghostface Killah/Supreme Clientele/07 The Grain.mp3
Finished with 7 of 328
/home/james/Music/Ghostface Killah/Supreme Clientele/08 Buck 50 (Ft. Method Man & Redman).mp3
Finished with 8 of 328
/home/james/Music/Ghostface Killah/Supreme Clientele/09 Mighty Healthy.mp3
Finished with 9 of 328
/home/james/Music/Ghostface Killah/Supreme Clientele/19 Wu Banga 101.mp3
Finished

Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed


Finished with 45 of 328
/home/james/Music/Ghostface Killah/The Wallabee Champ/16 Crockett & Tubbs f. Raekwon.mp3
Finished with 46 of 328
/home/james/Music/Ghostface Killah/The Wallabee Champ/14 Clips.mp3
Finished with 47 of 328
/home/james/Music/Ghostface Killah/The Wallabee Champ/03 ABC 1.mp3
Finished with 48 of 328
/home/james/Music/Ghostface Killah/The Wallabee Champ/10 We Dem Niggaz.mp3
Finished with 49 of 328
/home/james/Music/Ghostface Killah/The Wallabee Champ/08 Hidden Darts (Remix) 1.mp3
Finished with 50 of 328
/home/james/Music/Ghostface Killah/Face Of The Ghost  Bootleg/Pornstar (Feat F.T (Fuk That).mp3
Finished with 51 of 328
/home/james/Music/Ghostface Killah/Face Of The Ghost  Bootleg/Hideyaface (El-P Mix) (Feat El.mp3
Finished with 52 of 328
/home/james/Music/Ghostface Killah/Face Of The Ghost  Bootleg/Protect Ya Neck Instrumental I.mp3
Finished with 53 of 328
/home/james/Music/Ghostface Killah/Face Of The Ghost  Bootleg/Real Live Shit (Remix) (Feat R.mp3
Finished with 5

Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed
Lame tag CRC check failed


...

GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame doe

Finished with 228 of 328
/home/james/Music/City Lounge  New York Edition Vol. 1.www.lokotorrents.com/6. La Tendresse The Sura Quintet .mp3
Finished with 229 of 328
/home/james/Music/City Lounge  New York Edition Vol. 1.www.lokotorrents.com/2. Sensual Education - Enrico Donner.mp3
Finished with 230 of 328
/home/james/Music/City Lounge  New York Edition Vol. 1.www.lokotorrents.com/10. Love At Sunset - Solanos .mp3
Finished with 231 of 328
/home/james/Music/City Lounge  New York Edition Vol. 1.www.lokotorrents.com/5. Textpectations - Arrojas.mp3
Finished with 232 of 328
/home/james/Music/Black Star_Mos Def_Talib Kweli_Vinia Moj/Black Star/08 K. O. S. (Determination).m4a
Finished with 233 of 328
/home/james/Music/Ghostface Killah Feat. Trife Da God & Th/More Fish/11 Josephine.mp3
Finished with 234 of 328
/home/james/Music/City Lounge San Francisco Edition .2011[www.lokotorrents.com][mp3]/4 Enrico Donner - Lasting Memories.mp3
Finished with 235 of 328
/home/james/Music/City Lounge San Franc

GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type
GEOB frame does not contain a mime type
GEOB frame does not contain a valid mime type


Finished with 245 of 328
/home/james/Music/City Lounge San Francisco Edition .2011[www.lokotorrents.com][mp3]/5 Arrojas - Destinations.mp3
Finished with 246 of 328
/home/james/Music/City Lounge San Francisco Edition .2011[www.lokotorrents.com][mp3]/folder.jpg
Finished with 247 of 328
/home/james/Music/City Lounge San Francisco Edition .2011[www.lokotorrents.com][mp3]/10 Rey Salinero - Mystique Sets.mp3
Finished with 248 of 328
/home/james/Music/City Lounge San Francisco Edition .2011[www.lokotorrents.com][mp3]/1 Silent Breeze - Rain Clouds.mp3
Finished with 249 of 328
/home/james/Music/City Lounge San Francisco Edition .2011[www.lokotorrents.com][mp3]/13 Cane Garden Quartet - Chillaxin.mp3
Finished with 250 of 328
/home/james/Music/Ghostface Killah Feat. Redman & Shawn Wi/More Fish/10 Greedy Bitches.mp3
Finished with 251 of 328
/home/james/Music/Ghostface Killah Feat. Trife Da God & Su/More Fish/02 Miguel Sanchez.mp3
Finished with 252 of 328
/home/james/Music/City Lounge 8 - The Cool T

Finished with 317 of 328
/home/james/Music/City Lounge 8 - The Cool Tempo New Generation.2011[www.lokotorrents.com][mp3]/CD 2 - Paris/13. ALEX KEREN - Once Upon Time.mp3
Finished with 318 of 328
/home/james/Music/Black Star_Common_Mos Def_Talib Kweli/Black Star/11 Respiration.m4a
Finished with 319 of 328
/home/james/Music/Adrian Lux_Swedish House Mafia/Until One/18 Teenage Crime.mp3
Finished with 320 of 328
/home/james/Music/Steve Angello_Swedish House Mafia/Until One/19 Monday.mp3
Finished with 321 of 328
/home/james/Music/Steve Angello_Swedish House Mafia/Until One/13 Knas.mp3
Finished with 322 of 328
/home/james/Music/Ghostface Killah Feat. Kanye West & Ne-Y/More Fish/17 Back Like That (Remix).mp3
Finished with 323 of 328


In [56]:
from numpy import std
import re

# To make sure that we capture all characters in dir path strings from the dataframe
# Otherwise when it's converting with to_string(), it cuts off after X characters which is 
# converting the displayed output to a string but not the value itself. Stupid.
pd.options.display.max_colwidth = 1000


for i in list(dests.Album.unique()):
    print(i)    
    albs = dests[dests.Album == i]
    arts = albs.Artist.unique()
    if len(arts) > 1:
        print("More than one artist")
        ch_tbl = pd.DataFrame({"Artist": [], "Count" : [], "PCT": []})
        
        for t in arts:
            ch_art = t
            ch_nums = len(albs[albs.Artist == t])
            ch_pct = round((ch_nums/len(albs)) * 100, 1)
            ch_row = pd.DataFrame({"Artist": [ch_art], "Count" : [ch_nums], "PCT": [ch_pct]})
            ch_tbl = ch_tbl.append(ch_row)
        
        ## Catch Compilation Albums based on standard deviation
        
        if std(ch_tbl.PCT) < 3:     # Will want to figure out if 3 is the right thereshold.
            print("Compilation Album!")
            for t in arts:   
                old_loc = str(dests.New_Loc[(dests.Album == i) & (dests.Artist == t)].to_string(index=False))
                #print(t)
                #print("===============================")
                for line in [old_loc]:
                    
                    #print(str(line))
                    #print("++++++++++++++++\nNew Loc:")
                    new_loc = re.sub(pattern=str(t), repl="", string=line)
                    new_loc = str(re.sub(pattern="//", repl="/", string=new_loc))
                    #print(new_loc)
                    if line == new_loc:
                        print("Something ain't right here!")
                    else:
                        dests.New_Loc[(dests.New_Loc == line)] = new_loc
               # print("------------\n------------\n------------")
               # print(dests[(dests.Album == i) & (dests.Artist == t)])
               # print("------------\n------------\n------------")
        
        ## Replace folder naming if there are a few non-album artists in the album. ID3 tag stays the same
        
        else:
            rep = ch_tbl.sort_values(by=['PCT'],ascending=[False])
            rep = rep.Artist.iloc[0]         
            for t in arts:
                if t == rep:
                    continue
                else:
                    old_loc = str(dests.New_Loc[(dests.Album == i) & (dests.Artist == t)].to_string(index=False))
                    #print(t)
                    #print("===============================")
                    for line in [old_loc]:
                     #   print(str(line))
                      #  print("++++++++++++++++\nNew Loc:")
                        new_loc = re.sub(pattern=str(t), repl=rep, string=line)
                        #new_loc = str(re.sub(pattern="//", repl="/", string=new_loc))
                       # print(new_loc)
                        if line == new_loc:
                            print("Something ain't right here!")
                        else:
                            dests.New_Loc[(dests.New_Loc == line)] = new_loc
                    #print("------------\n------------\n------------")
                    #print(dests[(dests.Album == i) & (dests.Artist == t)])
                    #print("------------\n------------\n------------")        
    else:
        print("This looks just like an album!")
    print("-------------------------------------------")
    
print(deds)
    
    
    ## IF DIRPATH IS THE SAME/SIMILAR TO A GROUP OF SONGS, ASSIGN THE NON-SONG FILES TO THE NEW LOC FOR THOSE SONGS TOO


More Fish
This looks just like an album!
-------------------------------------------
Supreme Clientele
This looks just like an album!
-------------------------------------------
The Wallabee Champ
This looks just like an album!
-------------------------------------------
Face Of The Ghost  Bootleg
More than one artist
-------------------------------------------
Ghostdeini The Great
This looks just like an album!
-------------------------------------------
Hidden Darts Special Edition
This looks just like an album!
-------------------------------------------
Bulletproof Wallets
This looks just like an album!
-------------------------------------------
The RZA Hits
This looks just like an album!
-------------------------------------------
The Big Doe Rehab
This looks just like an album!
-------------------------------------------
Ghostdini Wizard Of Poetry In Emerald City
This looks just like an album!
-------------------------------------------
Ironman
This looks just like an album!
---

In [57]:
## Adding song destinations to all files table
## Go through all rows that have no destination (ie thumbs, etc.) and
## add new destination based on where everything else in that dirpath



# for g in dests.file_origin:
#     files.new_dir_path[files.full_path == g] = dests.New_Loc[dests.file_origin == g]
#     files.new_dir_name[files.full_path == g] = dests.New_Dir[dests.file_origin == g]

#files

## After this, you can go through and change things for both the thumbs and the deds in that folder!

for y in files.dir_path.unique():
    #get the intended new dir
    #check to see if there are NA lines
    ##if so, build new path and add new dir as well
    dest_dir = files.new_dir_name[(files.dir_path == y)].unique()
   # print(dest_dir)
    orphans = files[(files.dir_path == y) & (files.new_dir_path == 'NA') ]
    if not orphans.empty:       
        #for oliver in orphans:
        #    print(oliver)
        #    print("-----------")
        #files.new_dir_[(files.dir_path == y) & (files.new_dir_path == 'NA') ]
        print(orphans)
        
    #print(y)


                                                               dir_path dir_name    file_name                                                                         full_path new_dir_path new_dir_name
0  /home/james/Music/Ghostface Killah Feat. Trife Da God & Mr/More Fish       []  05 Good.mp3  /home/james/Music/Ghostface Killah Feat. Trife Da God & Mr/More Fish/05 Good.mp3           NA           NA
                                               dir_path dir_name                                 file_name                                                                                      full_path new_dir_path new_dir_name
0  /home/james/Music/Ghostface Killah/Supreme Clientele       []                         12 We Made It.mp3                         /home/james/Music/Ghostface Killah/Supreme Clientele/12 We Made It.mp3           NA           NA
0  /home/james/Music/Ghostface Killah/Supreme Clientele       []                 18 Cher Chez La Ghost.mp3                 /home/james/Music

0  /home/james/Music/Rudy_Swedish House Mafia_TV Rock/Until One       []  07 In The Air.mp3  /home/james/Music/Rudy_Swedish House Mafia_TV Rock/Until One/07 In The Air.mp3           NA           NA
                                                                      dir_path dir_name                                                            file_name                                                                                                                                        full_path new_dir_path new_dir_name
0  /home/james/Music/City Lounge  New York Edition Vol. 1.www.lokotorrents.com       []                   11. Everything Flows - Peter Linski Experience.mp3                   /home/james/Music/City Lounge  New York Edition Vol. 1.www.lokotorrents.com/11. Everything Flows - Peter Linski Experience.mp3           NA           NA
0  /home/james/Music/City Lounge  New York Edition Vol. 1.www.lokotorrents.com       []                                                           

In [52]:
from mutagen.mp4 import MP4
from mutagen.mp3 import MP3

#print(MP3('/home/james/Music/Ghostface Killah/Ghostdini Wizard Of Poetry In Emerald Ci/06 Stay.mp3').tags.get("artist"))

print(MP4('/home/james/Music/Black Star_Common_Mos Def_Talib Kweli/Black Star/11 Respiration.m4a').tags.get('\xa9alb')[0])

# '\xa9nam' – track title
# '\xa9alb' – album
# '\xa9ART' – artist
# 'aART' – album artist
# '\xa9wrt' – composer
# '\xa9day' – year
# '\xa9cmt' – comment









Black Star


In [ ]:
#os.path.join(art,alb)
for i in deds:
    ftt = id3.load(i)
    #print(dir(ftt))        
    #print(dir(ftt.tag))
    print("Album: {}".format(ftt.tag.album))
    print("Album Artist: {}".format(ftt.tag.album_artist))
    print("Album Type: {}".format(ftt.tag.album_type))
    print("Artist: {}".format(ftt.tag.artist))
    print("Artist Origin: {}".format(ftt.tag.artist_origin))
    print("Artist URL: {}".format(ftt.tag.artist_url))
    print("Audio File URL: {}".format(ftt.tag.audio_file_url))
    print("Audio Source URL: {}".format(ftt.tag.audio_source_url))
    print("Best Release Date: {}".format(ftt.tag.best_release_date))
    print("BPM: {}".format(ftt.tag.bpm))
    print("CD ID: {}".format(ftt.tag.cd_id))
    print("Chapters: {}".format(ftt.tag.chapters))
    print("Comments: {}".format(ftt.tag.comments))
    print("Commercial URL: {}".format(ftt.tag.commercial_url))
    print("Composer: {}".format(ftt.tag.composer))
    print("Copyright URL: {}".format(ftt.tag.copyright_url))
    print("Disk Num: {}".format(ftt.tag.disc_num))
    print("Encoding Date: {}".format(ftt.tag.encoding_date))
    print("Extended Header: {}".format(ftt.tag.extended_header))
    print("File Info: {}".format(ftt.tag.file_info))
    print("Frame Set: {}".format(ftt.tag.frame_set))
    print("Frameiter: {}".format(ftt.tag.frameiter))
    print("Genre: {}".format(ftt.tag.genre))
    print("Header: {}".format(ftt.tag.header))
    print("Images: {}".format(ftt.tag.images))
    print("Internet Radio URL: {}".format(ftt.tag.internet_radio_url))
    print("isV1: {}".format(ftt.tag.isV1))
    print("isV2: {}".format(ftt.tag.isV2))
    print("Lyrics: {}".format(ftt.tag.lyrics))
    print("Non Standard Genre: {}".format(ftt.tag.non_std_genre))
    print("Objects: {}".format(ftt.tag.objects))
    print("Original Release Date: {}".format(ftt.tag.original_release_date))
    print("Parse: {}".format(ftt.tag.parse))
    print("Payment_url: {}".format(ftt.tag.payment_url))
    print("Play Count: {}".format(ftt.tag.play_count))
    print("Popularities: {}".format(ftt.tag.popularities))
    print("Privates: {}".format(ftt.tag.privates))
    print("Publisher: {}".format(ftt.tag.publisher))
    print("Publisher URL: {}".format(ftt.tag.publisher_url))
    print("Read Only: {}".format(ftt.tag.read_only))
    print("Recording Date: {}".format(ftt.tag.recording_date))
    print("Release Date: {}".format(ftt.tag.release_date))
    print("SetTextFrame: {}".format(ftt.tag.setTextFrame))
    print("Table of Contents: {}".format(ftt.tag.table_of_contents))
    print("Tagging Date: {}".format(ftt.tag.tagging_date))
    print("Terms of Use: {}".format(ftt.tag.terms_of_use))
    print("Title: {}".format(ftt.tag.title))
    print("Track Number: {}".format(ftt.tag.track_num))
    print("Unique File Ids: {}".format(ftt.tag.unique_file_ids))
    print("User Text Frames: {}".format(ftt.tag.user_text_frames))
    print("User Url Frames: {}".format(ftt.tag.user_url_frames))
    print("Version: {}".format(ftt.tag.version))
    #break
    print("================\n===============")
    